In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Bidirectional, LSTM, Dense, Attention, Concatenate
from keras.models import Model
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import numpy as np

        

In [59]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.debugging.set_log_device_placement(False)

Num GPUs Available:  1


In [ ]:
df = pd.read_parquet('../data/btcusd_preprcd_with_regimes.parquet')
df = df[df['smoothed_regimes'] == 2]
df.dropna(inplace=True)

In [61]:
print(df.info())
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 257888 entries, 47 to 292662
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   open              257888 non-null  float64
 1   high              257888 non-null  float64
 2   low               257888 non-null  float64
 3   close             257888 non-null  float64
 4   BB_up_diff        257888 non-null  float64
 5   BB_down_diff      257888 non-null  float64
 6   OBV_Z             257888 non-null  float64
 7   MACD_Histogram    257888 non-null  float64
 8   RSI               257888 non-null  float64
 9   CCI               257888 non-null  float64
 10  predict_trend     257888 non-null  float64
 11  regime            257888 non-null  float64
 12  smoothed_regimes  257888 non-null  float64
dtypes: float64(13)
memory usage: 27.5 MB
None


open                0
high                0
low                 0
close               0
BB_up_diff          0
BB_down_diff        0
OBV_Z               0
MACD_Histogram      0
RSI                 0
CCI                 0
predict_trend       0
regime              0
smoothed_regimes    0
dtype: int64

In [62]:
def preprocess_data(train_df):
    # Log-transform prices
    for col in ['open', 'high', 'low', 'close']:
        train_df[f'{col}_log_return'] = np.log(train_df[col] / train_df[col].shift(1))
    
    # Robust scaling for Bollinger features
    robust_cols = ['BB_up_diff', 'BB_down_diff']
    robust_scaler = RobustScaler().fit(train_df[robust_cols])
    train_df[robust_cols] = robust_scaler.transform(train_df[robust_cols])
    
    # Standard scaling for MACD and CCI
    std_cols = ['MACD_Histogram', 'CCI']
    std_scaler = StandardScaler().fit(train_df[std_cols])
    train_df[std_cols] = std_scaler.transform(train_df[std_cols])
    
    # MinMax for RSI (focus on 30–70 range)
    train_df['RSI'] = np.clip(train_df['RSI'], 30, 70)
    rsi_scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_df[['RSI']])
    train_df['RSI'] = rsi_scaler.transform(train_df[['RSI']])
    
    train_df.drop(columns=['open', 'high', 'low', 'close', 'smoothed_regimes'], inplace=True)
    train_df.dropna(inplace=True)
    return train_df

In [63]:
df = preprocess_data(df)

In [64]:
df.head()

,BB_up_diff,BB_down_diff,OBV_Z,MACD_Histogram,RSI,CCI,predict_trend,regime,open_log_return,high_log_return,low_log_return,close_log_return
48,-0.328604,0.266544,0.753203,0.029921,0.993421,0.948745,-1.264000,1.0,0.001996,0.002316,0.000000,0.003951
49,-0.319143,0.272646,0.217083,0.027623,0.950845,0.638667,-0.025333,1.0,0.003410,0.007619,0.001037,-0.001819
50,-0.313345,0.276584,0.017523,0.023494,0.937636,0.458955,1.628667,1.0,0.000200,-0.005340,0.000974,-0.000988
51,-0.313166,0.277045,0.068341,0.020422,0.769865,0.495102,2.892000,1.0,-0.003008,-0.001663,-0.003080,0.000587
52,-0.315519,0.276394,0.083305,0.019048,0.721282,0.678630,4.018000,1.0,-0.000124,-0.000370,0.000000,0.001281


In [65]:
y = df['predict_trend']
x = df.drop(columns=['predict_trend'])
x_train = x[:int(len(x)*0.6)]
y_train = y[:int(len(x)*0.6)]
x_val = x[int(len(x)*0.6):int(len(x)*0.8)]
y_val = y[int(len(x)*0.6):int(len(x)*0.8)]
x_test = x[int(len(x)*0.8):]
y_test = y[int(len(x)*0.8):]

In [67]:
x_train = np.nan_to_num(x_train, nan=0.0, posinf=0.0, neginf=0.0)
y_train = np.nan_to_num(y_train, nan=0.0, posinf=0.0, neginf=0.0)
x_val = np.nan_to_num(x_val, nan=0.0, posinf=0.0, neginf=0.0)
y_val = np.nan_to_num(y_val, nan=0.0, posinf=0.0, neginf=0.0)

In [ ]:
from keras.regularizers import l2

# Define model parameters
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
time_steps = x_train.shape[1]
num_features = x_train.shape[2]
lstm_units = 64

# Define model architecture
inputs = Input(shape=(time_steps, num_features))
lstm_out = Bidirectional(LSTM(lstm_units, return_sequences=True, kernel_regularizer=l2(0.01)))(inputs)  # L2 regularization
attention = Attention()([lstm_out, lstm_out])
concat = Concatenate()([lstm_out, attention])
output = Dense(1, kernel_regularizer=l2(0.01))(concat)  # L2 regularization

model = Model(inputs=inputs, outputs=output)

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_val, y_val))


Epoch 1/5


2025-03-16 13:34:38.140377: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


1157/4836 ━━━━━━━━━━━━━━━━━━━━ 2:36 43ms/step - loss: 7964.4116